# **Notebook to fill the tables made for Algorhythm**

This notebook takes the data from the Spotify API with a library called Spotipy. Builds the functions to insert the data
and inserts it after calling the API methods for the data needed.


In [12]:
# Install required libraries
!uv add --group dev spotipy
!uv add --group dev python-dotenv

Resolved 155 packages in 5ms
Audited 149 packages in 0.18ms
Resolved 155 packages in 0.56ms
Audited 149 packages in 0.02ms


In [35]:
# Database setup and building script for SQLite
!python db/sqlite-setup.py

Database created at: ../../data/01_raw/algorhythms.db


### **Libraries**


In [ ]:
# Libraries
import os
import sqlite3
import secrets

# To prevent the API from being rate-limited
import time
import uuid
from datetime import datetime
from dataclasses import dataclass

import pandas as pd

# Spotify API setup
import spotipy

# Load environment variables
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth

In [37]:
# Load environment variables from .env file
load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI")

In [38]:
# Create a spotipy client with OAuth authentication

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope="user-library-read user-read-recently-played user-top-read",
    )
)

In [39]:
# Database connection
conn = sqlite3.connect("../../data/01_raw/algorhythms.db")
cursor = conn.cursor()

In [40]:
# Generate random user data
locations = ["Colombia", "Mexico", "Argentina", "Chile", "Brazil", "USA", "Spain", "Germany"]
genders = ["male", "female"]
music_profiles = [
    "Energetic, Latin Pop, Chill",
    "Classical, Instrumental, Calm",
    "Hip Hop, Trap, Urban",
    "Rock, Alternative, Indie",
    "Electronic, Dance, EDM",
    "Reggaeton, Latin, Party",
    "Jazz, Blues, Soul",
    "K-Pop, Pop, Trendy",
]

In [ ]:
# Create user profile
try:
    profile_data = sp.current_user()
    user_id = profile_data["id"]
except Exception as e:
    print(f"Could not get Spotify user profile: {e}")
    user_id = str(uuid.uuid4())  # Generate random user_id if not authenticated

age = secrets.randbelow(18) + 18  # [18, 35)
gender = secrets.choice(genders)
location = secrets.choice(locations)
profile = secrets.choice(music_profiles)

# Insert user if not already in the database
cursor.execute("SELECT 1 FROM User WHERE user_id = ?", (user_id,))
if cursor.fetchone() is None:
    cursor.execute(
        """
        INSERT INTO User (user_id, age, gender, location, music_profile)
        VALUES (?, ?, ?, ?, ?)
    """,
        (user_id, age, gender, location, profile),
    )
    print(f"Inserted new user: {user_id}")
else:
    print(f"User already exists: {user_id}")

conn.commit()

Inserted new user: 31rcpoje2ia746b5begjhg2lvzbi


In [ ]:
# 1. Function Definitions
def insert_artist(cursor, artist):
    cursor.execute(
        """
        INSERT OR IGNORE INTO Artist (artist_id, name, popularity, genres)
        VALUES (?, ?, ?, ?)
    """,
        (
            artist["id"],
            artist["name"],
            artist.get("popularity", 0),
            ", ".join(artist.get("genres", [])),
        ),
    )

In [21]:
def insert_album(cursor, sp, album, artist_id):
    album_details = sp.album(album["id"])
    cursor.execute(
        """
        INSERT OR IGNORE INTO Album (album_id, name, artist_id, release_date, popularity, genres)
        VALUES (?, ?, ?, ?, ?, ?)
    """,
        (
            album_details["id"],
            album_details["name"],
            artist_id,
            album_details.get("release_date", ""),
            album_details.get("popularity", 0),
            ", ".join(album_details.get("genres", [])),
        ),
    )

In [ ]:
@dataclass
class TrackInfo:
    track: dict
    artist_id: str
    album_id: str
    popularity: int
    duration_ms: int
    preview_url: str
    is_explicit: bool
    added_at: str | None  # fecha en que fue añadida a la biblioteca

def insert_track(cursor, sp, track_info: TrackInfo, user_id: str,
                 played_at: str | None = None, is_top=False, is_recent=False, is_liked=False):
    
    track = track_info.track
    track_id = track.get("id")
    if not track_id:
        print("Track without ID. Omitting...")
        return

    # Obtener géneros del artista
    genres = []
    try:
        artist_details = sp.artist(track_info.artist_id)
        genres = artist_details.get("genres", [])
    except Exception as e:
        print(f"Could not get genres for artist {track_info.artist_id}: {e}")

    # Obtener fecha de lanzamiento del álbum
    release_date = track.get("album", {}).get("release_date", "")

    # Insertar en la tabla Track
    cursor.execute(
        """
        INSERT OR IGNORE INTO Track (
            track_id, name, artist_id, album_id, popularity, genres, release_date,
            duration_ms, preview_url, is_explicit
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """,
        (
            track_id,
            track.get("name", "Unknown Track"),
            track_info.artist_id,
            track_info.album_id,
            track_info.popularity,
            ", ".join(genres),
            release_date,
            track_info.duration_ms,
            track_info.preview_url,
            int(track_info.is_explicit),
        )
    )

    # Insertar en UserTrackHistory si hay información del usuario
    if user_id:
        if not played_at:
            played_at = track_info.added_at or datetime.now().isoformat()

        cursor.execute(
            """
            INSERT OR IGNORE INTO UserTrackHistory (
                user_id, track_id, played_at, is_top, is_recent, is_liked
            )
            VALUES (?, ?, ?, ?, ?, ?)
            """,
            (
                user_id,
                track_id,
                played_at,
                int(is_top),
                int(is_recent),
                int(is_liked),
            )
        )

In [ ]:
# Top tracks
top_tracks = sp.current_user_top_tracks(limit=50, time_range="medium_term")["items"]
for track in top_tracks:
    artist = sp.artist(track["artists"][0]["id"])
    insert_artist(artist)
    insert_album(track["album"], artist["id"])
    insert_track(track["album"]["id"], artist["id"], is_top=True)
    time.sleep(0.2)

# Recent tracks
recent_tracks = sp.current_user_recently_played(limit=50)["items"]
for item in recent_tracks:
    track = item["track"]
    artist = sp.artist(track["artists"][0]["id"])
    insert_artist(artist)
    insert_album(track["album"], artist["id"])
    insert_track(track["album"]["id"], artist["id"], is_recent=True)
    time.sleep(0.2)

# Liked tracks (paginated)
limit = 50
offset = 0
while True:
    liked_page = sp.current_user_saved_tracks(limit=limit, offset=offset)["items"]
    if not liked_page:
        break
    for item in liked_page:
        track = item["track"]
        artist = sp.artist(track["artists"][0]["id"])
        insert_artist(artist)
        insert_album(track["album"], artist["id"])
        insert_track(track["album"]["id"], artist["id"], is_liked=True)
        time.sleep(0.2)  # Add delay to avoid rate limits
    offset += limit

conn.commit()

In [ ]:
# Ruta a la base de datos
db_path = "../../data/01_raw/algorhythms.db"
conn = sqlite3.connect(db_path)

tables = ["User", "Artist", "Album", "Track", "UserTrackHistory"]
output_folder = os.path.dirname(db_path)

# Exportar tablas a CSV
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM [{table}]", conn)
    file_path = os.path.join(output_folder, f"{table}.csv")
    df.to_csv(file_path, index=False)

conn.close()

# Cargar CSVs en diccionario
df_dict = {}
for file in os.listdir(output_folder):
    if file.endswith(".csv"):
        name = file[:-4]  # elimina ".csv"
        file_path = os.path.join(output_folder, file)
        df_dict[name] = pd.read_csv(file_path)

# Asignar DataFrames a variables globales
globals().update(df_dict)

In [ ]:
# Ruta a la base de datos
db_path = "../../data/01_raw/algorhythms.db"

# Validar y preparar conexión a la base de datos
if db_path is not None:
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = sqlite3.connect(db_path)
else:
    raise ValueError("Database path cannot be None")

# Tablas seguras para exportar
safe_tables = {"User", "Artist", "Album", "Track", "UserTrackHistory"}

# Carpeta para guardar archivos CSV
output_folder = os.path.dirname(db_path)
os.makedirs(output_folder, exist_ok=True)

# Exportar tablas a CSV
for table in safe_tables:
    try:
        df = pd.read_sql_query(f"SELECT * FROM [{table}]", conn)
        file_path = os.path.join(output_folder, f"{table}.csv")
        df.to_csv(file_path, index=False)
        print(f"{table}.csv guardado en: {file_path}")
    except Exception as e:
        print(f"Error al exportar {table}: {e}")

# Cerrar la conexión
conn.close()

User.csv guardado en: ../../data/01_raw/User.csv
Artist.csv guardado en: ../../data/01_raw/Artist.csv
Album.csv guardado en: ../../data/01_raw/Album.csv
Track.csv guardado en: ../../data/01_raw/Track.csv
UserTrackHistory.csv guardado en: ../../data/01_raw/UserTrackHistory.csv


In [ ]:
import os
import pandas as pd

folder_path = "../../data/01_raw"
dataframes = {}

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        name = os.path.splitext(filename)[0]
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            dataframes[name] = df
            print(f"Dataframe: {name} ({df.shape[0]} rows)")
        except Exception as e:
            print(f"Error loading {filename}: {e}")

Dataframe: Artist (612 rows)
Dataframe: Album (1390 rows)
Dataframe: Track (2019 rows)
Dataframe: UserTrackHistory (2336 rows)
Dataframe: User (500 rows)


In [25]:
# Display the first rows of each DataFrame
for name, df in dataframes.items():
    print(f"{name} DataFrame:")
    display(df.head(3))

Artist DataFrame:


,artist_id,name,genres,popularity
0,4tuJ0bMpJh08umKkEXKUI5,Gracie Abrams,NaN,87
1,06HL4z0CvFAxyc27GXpf02,Taylor Swift,NaN,98
2,2sSGPbdZJkaSE2AbcGOACx,The Marías,bedroom pop,86


Album DataFrame:


,album_id,name,artist_id,release_date,popularity,genres
0,4XXTsu7r9865VvXdvF2iQP,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-20,0,NaN
1,56bdWeO40o3WfAD2Lja4dl,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-21,0,NaN
2,1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,06HL4z0CvFAxyc27GXpf02,2024-04-18,0,NaN


Track DataFrame:


,track_id,name,artist_id,album_id,popularity,genres,features_vector,release_date
0,6nN8W5zHOii0P61I8eSdR3,Free Now,4tuJ0bMpJh08umKkEXKUI5,4XXTsu7r9865VvXdvF2iQP,69,NaN,NaN,2024-06-20
1,51rfRCiUSvxXlCSCfIztBy,"I Love You, I'm Sorry",4tuJ0bMpJh08umKkEXKUI5,56bdWeO40o3WfAD2Lja4dl,90,NaN,NaN,2024-06-21
2,5wbg8kepMFoMzHOEuxiI0q,Close To You,4tuJ0bMpJh08umKkEXKUI5,4XXTsu7r9865VvXdvF2iQP,85,NaN,NaN,2024-06-20


UserTrackHistory DataFrame:


,user_id,track_id,played_at,is_top_track,is_recent_play,is_liked
0,user_123,6nN8W5zHOii0P61I8eSdR3,2025-05-28T07:20:47.132908,1,0,0
1,user_123,51rfRCiUSvxXlCSCfIztBy,2025-05-28T07:20:47.270594,1,0,0
2,user_123,5wbg8kepMFoMzHOEuxiI0q,2025-05-28T07:20:47.386019,1,0,0


User DataFrame:


,user_id,age,gender,location,music_profile
0,fd9e55bf-2d62-4fd2-8f10-065b269d7d6c,18,non-binary,Mexico,"Electronic, Dance, EDM"
1,bbba773c-7c30-45fa-b440-4ab5acd0544e,20,female,USA,"Rock, Alternative, Indie"
2,76eacce5-d66f-4529-8d21-15d0afe4b0ed,23,male,USA,"Hip Hop, Trap, Urban"


In [26]:
def insert_chart_track(
    cursor, track, artist_id, album_id, chart_id, chart_name, position, added_at=None
):
    genres = []
    try:
        artist_details = sp.artist(artist_id)
        genres = artist_details.get("genres", [])
    except Exception as e:
        print(f"Could not get genres for artist {artist_id}: {e}")

    release_date = track["album"].get("release_date", "") if track.get("album") else ""

    cursor.execute(
        """
        INSERT OR IGNORE INTO ChartTrack (
            chart_id, track_id, name, artist_id, album_id, popularity, genres, release_date, chart_name, position, added_at
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
        (
            chart_id,
            track.get("id"),
            track.get("name", "Unknown Track"),
            artist_id,
            album_id,
            track.get("popularity", 0),
            ", ".join(genres),
            release_date,
            chart_name,
            position,
            added_at,
        ),
    )

In [ ]:
def search_playlist_id(sp, query, limit=10):
    results = sp.search(q=query, type="playlist", limit=limit)
    items = results.get("playlists", {}).get("items", [])
    for playlist in items:
        if playlist is not None and playlist.get("public", False):
            return {"name": playlist["name"], "id": playlist["id"]}
    return None


def get_top_genre_and_daily_playlists(sp):
    top_playlists = {
        "Today's Top Hits": "Today's Top Hits",
        "Top 50 Global": "Top 50 Global",
    }

    countries = ["Colombia", "Mexico", "Argentina", "Chile", "Brazil", "USA", "Spain", "Germany"]

    for country in countries:
        top_playlists[f"Top 50 {country}"] = f"Top 50 {country}"

    results = {}

    for key, query in top_playlists.items():
        playlist = search_playlist_id(sp, query)
        results[key] = playlist["id"] if playlist else None

    return results


# Run the function and display results
playlist_ids = get_top_genre_and_daily_playlists(sp)
playlist_ids

{"Today's Top Hits": '4Jb4PDWREzNnbZcOHPcZPy',
 'Top 50 Global': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Colombia': '6h6uzoRBXnkjeoEjwiX27R',
 'Top 50 Mexico': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Argentina': '3NJmFR02JQRYIB7COYckSR',
 'Top 50 Chile': '6lS871B4ot8ALAb0axxRTX',
 'Top 50 Brazil': '3UNpjQZyOI1ma6rEOkeoyw',
 'Top 50 USA': '6h6uzoRBXnkjeoEjwiX27R',
 'Top 50 Spain': '7ki8VipxMygh7Y8ZdPRvmX',
 'Top 50 Germany': '49LS36Wz4Mu2fX2M63ma7M'}

In [ ]:
# import time

# chart_name = "Top 50 Mexico"
# chart_id = '3NJmFR02JQRYIB7COYckSR'

# try:
#     limit = 100
#     offset = 0
#     position = 1
#     while True:
#         response = sp.playlist_items(chart_id, limit=limit, offset=offset)
#         items = response['items']
#         for item in items:
#             track = item['track']
#             artist_id = track['artists'][0]['id']
#             album_id = track['album']['id']
#             added_at = item.get('added_at', None)
#             insert_artist(cursor, sp.artist(artist_id))
#             insert_album(cursor, sp, track['album'], artist_id)
#             insert_track(cursor, sp, track, album_id, artist_id, user_id=None)
#             insert_chart_track(cursor, track, artist_id, album_id, chart_id, chart_name, position, added_at)
#             position += 1
#             time.sleep(0.1)  # Reduce this si ya no necesitas evitar rate limits
#         if response['next'] is None:
#             break
#         offset += limit
# except Exception as e:
#     print(f"Error processing playlist {chart_name} ({chart_id}): {e}")

# conn.commit()

KeyboardInterrupt: 

In [49]:
# Diccionario: tabla → claves primarias
tables = {
    "User": ["user_id"],
    "Artist": ["artist_id"],
    "Album": ["album_id"],
    "Track": ["track_id"],
    "UserTrackHistory": ["user_id", "track_id", "played_at"],
    "ChartTrack": ["chart_id", "track_id"],
}

# Carpeta para guardar los CSVs
output_folder = "../../data/01_raw"
os.makedirs(output_folder, exist_ok=True)

# Exportar cada tabla a CSV, evitando duplicados
for table, primary_keys in tables.items():
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    file_path = os.path.join(output_folder, f"{table}.csv")

    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)

        # Concatenar y eliminar duplicados con base en claves primarias
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        combined_df.drop_duplicates(subset=primary_keys, keep="last", inplace=True)
    else:
        combined_df = df

    # Guardar archivo actualizado
    combined_df.to_csv(file_path, index=False)
    print(f"{table}.csv actualizado en: {file_path}")

User.csv actualizado en: ../../data/01_raw/User.csv
Artist.csv actualizado en: ../../data/01_raw/Artist.csv
Album.csv actualizado en: ../../data/01_raw/Album.csv
Track.csv actualizado en: ../../data/01_raw/Track.csv
UserTrackHistory.csv actualizado en: ../../data/01_raw/UserTrackHistory.csv
ChartTrack.csv actualizado en: ../../data/01_raw/ChartTrack.csv


In [ ]:
# # For each playlist in playlist_ids, insert all tracks into ChartTrack with added_at and pagination
# for chart_name, chart_id in playlist_ids.items():
#     if not chart_id:
#         continue
#     try:
#         limit = 100
#         offset = 0
#         position = 1
#         while True:
#             response = sp.playlist_items(chart_id, limit=limit, offset=offset)
#             items = response['items']
#             for item in items:
#                 track = item['track']
#                 artist_id = track['artists'][0]['id']
#                 album_id = track['album']['id']
#                 added_at = item.get('added_at', None)
#                 insert_artist(cursor, sp.artist(artist_id))
#                 insert_album(cursor, sp, track['album'], artist_id)
#                 insert_track(cursor, sp, track, album_id, artist_id, user_id=None)
#                 insert_chart_track(cursor, track, artist_id, album_id, chart_id, chart_name, position, added_at)
#                 position += 1
#                 time.sleep(0.2)
#             if response['next'] is None:
#                 break
#             offset += limit
#     except Exception as e:
#         print(f"Error processing playlist {chart_name} ({chart_id}): {e}")

# conn.commit()

KeyboardInterrupt: 

In [ ]:
# Close the database connection after all inserts
conn.close()

In [53]:
# Folder path for CSV files
folder_path = "../../data/01_raw"

dataframes = {}

# Load CSV files from the specified folder into a dictionary of DataFrames
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        name = os.path.splitext(filename)[0]
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes[name] = df
        print(f"Dataframe: {name} ({df.shape[0]} rows)")

Dataframe: Artist (748 rows)
Dataframe: Album (1717 rows)
Dataframe: Track (2416 rows)
Dataframe: UserTrackHistory (3052 rows)
Dataframe: User (501 rows)
Dataframe: ChartTrack (473 rows)


In [54]:
# Display the first rows of each DataFrame
for name, df in dataframes.items():
    print(f"{name} DataFrame:")
    display(df.head(3))

Artist DataFrame:


,artist_id,name,genres,popularity
0,4tuJ0bMpJh08umKkEXKUI5,Gracie Abrams,NaN,87
1,2sSGPbdZJkaSE2AbcGOACx,The Marías,bedroom pop,86
2,4Uc8Dsxct0oMqx0P6i60ea,Conan Gray,NaN,80


Album DataFrame:


,album_id,name,artist_id,release_date,popularity,genres
0,4XXTsu7r9865VvXdvF2iQP,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-20,0,NaN
1,56bdWeO40o3WfAD2Lja4dl,The Secret of Us,4tuJ0bMpJh08umKkEXKUI5,2024-06-21,0,NaN
2,1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,06HL4z0CvFAxyc27GXpf02,2024-04-18,0,NaN


Track DataFrame:


...

,user_id,track_id,played_at,is_top_track,is_recent_play,is_liked,is_top,is_recent
0,user_123,6nN8W5zHOii0P61I8eSdR3,2025-05-28T07:20:47.132908,1.0,0.0,0,NaN,NaN
1,user_123,51rfRCiUSvxXlCSCfIztBy,2025-05-28T07:20:47.270594,1.0,0.0,0,NaN,NaN
2,user_123,5wbg8kepMFoMzHOEuxiI0q,2025-05-28T07:20:47.386019,1.0,0.0,0,NaN,NaN


User DataFrame:


,user_id,age,gender,location,music_profile
0,fd9e55bf-2d62-4fd2-8f10-065b269d7d6c,18,non-binary,Mexico,"Electronic, Dance, EDM"
1,bbba773c-7c30-45fa-b440-4ab5acd0544e,20,female,USA,"Rock, Alternative, Indie"
2,76eacce5-d66f-4529-8d21-15d0afe4b0ed,23,male,USA,"Hip Hop, Trap, Urban"


ChartTrack DataFrame:


,chart_id,track_id,name,artist_id,album_id,popularity,genres,release_date,chart_name,position,added_at
0,4Jb4PDWREzNnbZcOHPcZPy,5IZXB5IKAD2qlvTPJYDCFB,I Had Some Help (Feat. Morgan Wallen),246dkjvS1zLTtiykXe5h60,4BbsHmXEghoPPevQjPnHXx,88,NaN,2024-08-15,Today's Top Hits,1,2024-10-23T15:33:22Z
1,4Jb4PDWREzNnbZcOHPcZPy,2uqYupMHANxnwgeiXTZXzd,Austin (Boots Stop Workin'),7Ez6lTtSMjMf2YSYpukP1I,40HsqPqeSR9Xe3IyAJWr6e,88,NaN,2024-02-16,Today's Top Hits,2,2024-05-09T22:50:38Z
2,4Jb4PDWREzNnbZcOHPcZPy,3Rfre3qkrhwdZZ7dyznwbN,Lonely Road (with Jelly Roll),6TIYQ3jFPwQSRmorSezPxX,4tU0FNnuiBD1P6IRTARHww,79,NaN,2024-07-26,Today's Top Hits,3,2024-10-23T14:37:11Z
